In [1]:
from Code.general_functions import *
place = 'Turin_Italy'
my_preprocessing = Preprocessing(place,is_test=True)
data_folder, data_test = my_preprocessing.create_folder()

Data folder: C:\Users\18059\OneDrive - ariel.ac.il\Current_research\SOD\Code\places/Turin_Italy
Data folder: C:\Users\18059\OneDrive - ariel.ac.il\Current_research\SOD\Code\places/Turin_Italy/test


In [2]:
data_test

'C:\\Users\\18059\\OneDrive - ariel.ac.il\\Current_research\\SOD\\Code\\places/Turin_Italy/test'

In [2]:

data_test_topology = f'{data_test}/topology'
new_network  = gpd.read_file(f'{data_folder}/simp.shp')
num = 0
new_gpd = new_network.copy()
obj_intersection = Intersection(new_gpd, num)
obj_intersection.my_network.to_file(f'{data_test_topology}/topology_0__before.shp')
obj_intersection.intersection_network()
obj_intersection.my_network.to_file(f'{data_test_topology}/topology_0_intersection.shp')
obj_intersection.update_names(new_gpd)
obj_intersection.my_network.to_file(f'{data_test_topology}/topology_0.shp')

Update topology


100%|██████████| 1948/1948 [00:25<00:00, 76.26it/s]


In [3]:
my_gdf =  gpd.read_file(os.path.join(data_folder, 'osm_data.gpkg'), layer='edges')
round_about = my_gdf[my_gdf['junction'].isin(['roundabout', 'circular'])]
round_about

,u,v,key,osmid,oneway,name,highway,reversed,length,bearing,...,lanes,access,bridge,maxspeed,junction,service,tunnel,width,ref,geometry
110,13929741,248847593,0,713890756,True,Piazza Rivoli,primary,False,8.383,25.0,...,3,,,50,roundabout,,,,,"LINESTRING (850888.050 5633392.778, 850893.081..."
112,13929741,248151527,0,713797731,True,Piazza Rivoli,primary,False,18.931,55.0,...,3,,,50,roundabout,,,,,"LINESTRING (850866.209 5633377.503, 850877.085..."
113,248847593,427877534,0,713890757,True,Piazza Rivoli,primary,False,5.968,6.7,...,3,,,50,roundabout,,,,,"LINESTRING (850893.081 5633403.545, 850894.072..."
441,28349001,28349002,0,160587553,True,Piazza Marmolada,tertiary,False,39.360,124.8,...,2,,,50,roundabout,,,,,"LINESTRING (851457.460 5630129.243, 851461.501..."
442,28349001,446773120,0,170594843,True,Piazza Marmolada,tertiary,False,24.322,197.9,...,2,,,50,roundabout,,,,,"LINESTRING (851467.345 5630159.782, 851462.436..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55070,11959561646,11959561664,0,1289794343,True,,service,False,5.114,198.5,...,,,,,roundabout,,,,,"LINESTRING (859413.787 5640589.023, 859411.861..."
55071,11959561658,11959561657,0,1289794343,True,,service,False,3.294,106.3,...,,,,,roundabout,,,,,"LINESTRING (859420.856 5640565.217, 859425.342..."
55072,11959599174,11959561667,0,1289794343,True,,service,False,23.075,335.3,...,,,,,roundabout,,,,,"LINESTRING (859441.483 5640571.338, 859443.420..."
55073,11959599174,11959561657,0,1289794343,True,,service,False,13.190,65.3,...,,,,,roundabout,,,,,"LINESTRING (859425.342 5640563.907, 859430.028..."


In [ ]:
line_name = 'line_name'
if my_preprocessing.is_junction:
    print('Update roundabout')
    exist_data = obj_intersection.my_network.reset_index().reset_index(names=line_name)

    my_roundabout = Roundabout(exist_data, round_about)
    deadend_lines, deadend_pnts = my_roundabout.deadend()

    # update the current network
    change_geo = my_roundabout.my_spatial_join(deadend_lines, deadend_pnts, line_name)
    my_roundabout.update_the_current_network(change_geo)

    my_roundabout.network.drop_duplicates(subset=line_name, inplace=True)
    # Improve roundabout
    # First buffer around centroid
    centr_name = 'centr_name'
    buffer_around_centroid = my_roundabout.centroid['geometry'].buffer(cap_style=1, distance=30)

    # s_join between buffer to lines (reset index to retain the original centroid name which can apper more than one in the results). always stay with data you need and with understandable name
    roundabout_with_lines = \
        gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_centroid, crs=project_crs).reset_index(),
                  right_df=my_roundabout.network[['geometry', line_name]]).drop_duplicates(
            subset=['index', line_name]).rename(columns={"index": centr_name})[['geometry', line_name, centr_name]]

    # To facilitate the searching process
    my_roundabout.network.set_index(line_name, inplace=True)
    # To facilitate easy access to point centroid geometry data, it is advisable to store the information in an object that provides efficient retrieval.
    pnt_centroid_temp = my_roundabout.centroid['geometry']
    #  Group the data by centroid
    for center_line in roundabout_with_lines.groupby(centr_name):
        #  Iterate over each group after performing a groupby() operation
        for center in center_line[1].itertuples():
            # Find the line that connects to the current centroid and obtain its vertices
            line_to_test = my_roundabout.network.loc[center[2]]
            vertices_line = list(line_to_test['geometry'].coords)
            pnt_test = [vertices_line[0], vertices_line[-1]]
            # To determine if the current line is already connected to the current centroid.
            is_connected = my_roundabout.centroid[
                my_roundabout.centroid['geometry'].isin([Point(pnt_test[0]), Point(pnt_test[-1])])]
            if len(is_connected) > 0 and center[3] in is_connected['name']:
                continue

            if len(vertices_line) == 2:
                vertices_line.insert(1, pnt_centroid_temp[center[3]])
            else:
                my_list = [pnt_centroid_temp[center[3]].distance(Point(temp)) for temp in vertices_line]
                # Find the minimum index
                min_index = min(range(len(my_list)), key=my_list.__getitem__)
                if min_index == 0:
                    vertices_line.insert(0, pnt_centroid_temp[center[3]])
                elif min_index == len(my_list) - 1:
                    vertices_line.append(pnt_centroid_temp[center[3]])
                else:
                    vertices_line[min_index] = pnt_centroid_temp[center[3]]
            new_geo = LineString(vertices_line)
            my_roundabout.network.at[center[2], 'geometry'] = new_geo

    new_network1 = my_roundabout.network.reset_index()
    new_network1.drop(columns='index', inplace=True)


    # Function to remove self-intersecting LineString geometries
    def remove_self_intersecting(line):
        return line.is_simple


    # Apply the function to filter out self-intersecting geometries
    new_network2 = new_network1[new_network1['geometry'].apply(remove_self_intersecting)]
else:
    new_network2 = obj_intersection.my_network.reset_index()


In [5]:

extend_lines_f = extend_lines(new_network2, 100)
extend_lines_f['length'] = extend_lines_f.length
obj_intersection_1 = Intersection(extend_lines_f.copy(), 1)
obj_intersection_1.intersection_network()
obj_intersection_1.my_network.rename(columns={'str_name': 'name'},
                                     inplace=True)  # 'str_name' become 'str to be compatible with other previous networks
obj_intersection_1.update_names(org_gpd=extend_lines_f.copy().rename(columns={'str_name': 'name'}))



# Clear short segments
final2 = EnvEntity(obj_intersection_1.my_network.reset_index())
final2.update_the_current_network(final2.get_deadend_gdf(delete_short=100))
final2.network.drop(columns=[line_name, 'bearing']).to_file(f'{data_test}/network_test.shp')
